# ACTA 5. Preprocesado de los datos. PARTE 2

## Introducción

El propósito de esta segunda parte del peprocesado es el de preparar nuestra base de datos para poder emplearla en la obtención de los modelos estadísticos. Lo que haremos será codificar las variables del conjunto de datos resultante de la aplicación del documento ACTA 3. Limpieza y análisis exploratorio de los datos. PARTE 2. 

Para ello comenzaremos cargando nuestra base de datos. 

In [1]:
import pandas as pd

Incidents = pd.read_csv('Incidents_SF.csv')

Incidents.head()

,incident_datetime,incident_day_of_week,incident_code,incident_category,incident_subcategory,resolution,latitude,longitude,neighborhood,Police_district,Holiday,Street
0,2020-06-22 08:00:00,Monday,9029,FRAUD,FRAUD,Open or Active,37.772618,-122.483119,Outer Richmond,RICHMOND,No,FULTON ST
1,2020-06-22 13:30:00,Monday,7043,RECOVERED VEHICLE,RECOVERED VEHICLE,Open or Active,37.765117,-122.418579,Mission,MISSION,No,16TH ST
2,2020-06-22 22:26:00,Monday,12026,ASSAULT,AGGRAVATED ASSAULT,Open or Active,37.728942,-122.378542,Bayview Hunters Point,BAYVIEW,No,NAVY RD
3,2020-06-22 18:00:00,Monday,6243,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.712320,-122.456045,Oceanview/Merced/Ingleside,TARAVAL,No,SADOWA ST
4,2020-06-22 11:30:00,Monday,6244,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.806658,-122.420560,Russian Hill,CENTRAL,No,HYDE ST


In [2]:
missing_values_count = Incidents.isna().sum()
print(missing_values_count)

incident_datetime       0
incident_day_of_week    0
incident_code           0
incident_category       0
incident_subcategory    0
resolution              0
latitude                0
longitude               0
neighborhood            0
Police_district         0
Holiday                 0
Street                  0
dtype: int64


In [3]:
Incidents["incident_datetime"] = pd.to_datetime(Incidents["incident_datetime"])
Incidents["day"] = Incidents["incident_datetime"].dt.day
Incidents["month"] = Incidents["incident_datetime"].dt.month
Incidents["year"] = Incidents["incident_datetime"].dt.year
Incidents["hour"] = Incidents["incident_datetime"].dt.hour
Incidents["minutes"] = Incidents["incident_datetime"].dt.minute

Antes de codificar nuestras variables, vamos a transformar la columna 'incident_datetime', esta variable contiene información relevante y para poder incluirla en el modelo y hacer un buen estudio vamos a crear las columnas, 'day', 'month', 'year' y 'minutes' y dejamos la clumna 'incident_datetime' para conservar la original. 

In [4]:
##CREACIÓN DE COLUMNAS
def encode_week(day):
    if day <= 7:
        return 'week1'
    elif day <= 14:
        return 'week2'
    elif day <= 21:
        return 'week3'
    else:
        return 'week4'

def encode_quarter(month):
    if month in [1, 2, 3]:
        return 'quarter1'
    elif month in [4, 5, 6]:
        return 'quarter2'
    elif month in [7, 8, 9]:
        return 'quarter3'
    else:
        return 'quarter4'

def encode_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

def encode_hour(hour):
    if hour < 4:
        return '0-4'
    elif hour < 8:
        return '4-8'
    elif hour < 12:
        return '8-12'
    elif hour < 16:
        return '12-16'
    elif hour < 20:
        return '16-20'
    else:
        return '20-24'

def encode_minutes(minutes):
    if minutes < 15:
        return '0-15'
    elif minutes < 30:
        return '15-30'
    elif minutes < 45:
        return '30-45'
    else:
        return '45-60'

Incidents["week"] = Incidents["day"].apply(encode_week)
Incidents["quarter"] = Incidents["month"].apply(encode_quarter)
Incidents["season"] = Incidents["month"].apply(encode_season)
Incidents["interval_hour"] = Incidents["hour"].apply(encode_hour)
Incidents["interval_minutes"] = Incidents["minutes"].apply(encode_minutes)

print('-----------------dayVsWeek--------------------')
print(Incidents[["day", "week"]].head(10))
print('-----------------monthVsquarter--------------------')
print(Incidents[["month", "quarter"]].head(10))
print('-----------------monthVsseason--------------------')
print(Incidents[["month", "season"]].head(10))
print('-----------------hourVsinterval_hour--------------------')
print(Incidents[["hour", "interval_hour"]].head(10))
print('-----------------minutesVinterval_minutes--------------------')
print(Incidents[["minutes", "interval_minutes"]].head(10))


-----------------dayVsWeek--------------------
   day   week
0   22  week4
1   22  week4
2   22  week4
3   22  week4
4   22  week4
5   22  week4
6   22  week4
7   22  week4
8   22  week4
9   22  week4
-----------------monthVsquarter--------------------
   month   quarter
0      6  quarter2
1      6  quarter2
2      6  quarter2
3      6  quarter2
4      6  quarter2
5      6  quarter2
6      6  quarter2
7      6  quarter2
8      6  quarter2
9      6  quarter2
-----------------monthVsseason--------------------
   month  season
0      6  summer
1      6  summer
2      6  summer
3      6  summer
4      6  summer
5      6  summer
6      6  summer
7      6  summer
8      6  summer
9      6  summer
-----------------hourVsinterval_hour--------------------
   hour interval_hour
0     8          8-12
1    13         12-16
2    22         20-24
3    18         16-20
4    11          8-12
5    16         16-20
6    15         12-16
7    18         16-20
8    22         20-24
9    17         16-20
-

## Variables Codificadas

A continuación crearemos las variables la codificación de variables para añadir a nuestro conjunto de datos inicial las variables codificadas. 
Para ello emplearemos la función encode_categorical_variables para que nos añada a nuestro dataframe las variables codificadas pero además nos devuelva un diccionario con un diccionario para cada variable con la correspondencia entre los valores originales y los valores originales codifcados. 

In [5]:
def encode_categorical_variables(df, cols):
    val_dict = {}
    for col in cols:
        unique_values = df[col].unique()
        val_dict[col] = {val: i for i, val in enumerate(unique_values)}
        df[col + "_cod"] = df[col].map(val_dict[col])
    return df, val_dict


Aplicamos nuestra codificación sobre nuestro conjunto de datos excluyendo las variables 'incident_code', 'latitude', 'longitude' porque la primera ya está codificada y la segunda y tercera ya son variables numéricas. 

In [6]:

cols_to_encode = [col for col in Incidents.columns if col not in ['incident_code', 'latitude', 'longitude','incident_datetime','day','month','year','hour','minutes']]


Incidents_F, val_dict = encode_categorical_variables(Incidents, cols_to_encode)


print(Incidents_F.head())


    incident_datetime incident_day_of_week  incident_code  incident_category   
0 2020-06-22 08:00:00               Monday           9029              FRAUD  \
1 2020-06-22 13:30:00               Monday           7043  RECOVERED VEHICLE   
2 2020-06-22 22:26:00               Monday          12026            ASSAULT   
3 2020-06-22 18:00:00               Monday           6243      LARCENY THEFT   
4 2020-06-22 11:30:00               Monday           6244      LARCENY THEFT   

     incident_subcategory      resolution   latitude   longitude   
0                   FRAUD  Open or Active  37.772618 -122.483119  \
1       RECOVERED VEHICLE  Open or Active  37.765117 -122.418579   
2      AGGRAVATED ASSAULT  Open or Active  37.728942 -122.378542   
3  LARCENY - FROM VEHICLE  Open or Active  37.712320 -122.456045   
4  LARCENY - FROM VEHICLE  Open or Active  37.806658 -122.420560   

                 neighborhood Police_district  ... resolution_cod   
0              Outer Richmond        RICH

Visulizamos nuestro conjunto de datos para comprobar que hemos conseguido codificar nuestras variables. 

In [7]:
Incidents_F.head()

,incident_datetime,incident_day_of_week,incident_code,incident_category,incident_subcategory,resolution,latitude,longitude,neighborhood,Police_district,...,resolution_cod,neighborhood_cod,Police_district_cod,Holiday_cod,Street_cod,week_cod,quarter_cod,season_cod,interval_hour_cod,interval_minutes_cod
0,2020-06-22 08:00:00,Monday,9029,FRAUD,FRAUD,Open or Active,37.772618,-122.483119,Outer Richmond,RICHMOND,...,0,0,0,0,0,0,0,0,0,0
1,2020-06-22 13:30:00,Monday,7043,RECOVERED VEHICLE,RECOVERED VEHICLE,Open or Active,37.765117,-122.418579,Mission,MISSION,...,0,1,1,0,1,0,0,0,1,1
2,2020-06-22 22:26:00,Monday,12026,ASSAULT,AGGRAVATED ASSAULT,Open or Active,37.728942,-122.378542,Bayview Hunters Point,BAYVIEW,...,0,2,2,0,2,0,0,0,2,2
3,2020-06-22 18:00:00,Monday,6243,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.712320,-122.456045,Oceanview/Merced/Ingleside,TARAVAL,...,0,3,3,0,3,0,0,0,3,0
4,2020-06-22 11:30:00,Monday,6244,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.806658,-122.420560,Russian Hill,CENTRAL,...,0,4,4,0,4,0,0,0,0,1


In [8]:
Incidents_F.resolution_cod.value_counts()

resolution_cod
0    536821
1    135708
2      3753
3      1843
Name: count, dtype: int64

In [9]:
Incidents_F.Holiday_cod.value_counts()

Holiday_cod
0    650129
1     27996
Name: count, dtype: int64

In [10]:
Incidents_F.week_cod.value_counts()

week_cod
0    207306
1    161131
2    154957
3    154731
Name: count, dtype: int64

In [11]:
Incidents_F.quarter_cod.value_counts()

quarter_cod
3    187144
1    168301
2    164718
0    157962
Name: count, dtype: int64

In [12]:
Incidents_F.season_cod.value_counts()

season_cod
2    179462
3    166820
1    166158
0    165685
Name: count, dtype: int64

Ahora vamos a mostrar como ejemplo la obtención de la correspondencia entre los valores originales de nuestra variable y los valores codificados.

In [13]:
print("Valores originales de 'resolution':")
print(val_dict['resolution'])

Valores originales de 'resolution':
{'Open or Active': 0, 'Cite or Arrest Adult': 1, 'Unfounded': 2, 'Exceptional Adult': 3}


Ahora guardamos  el diccionario con los valores para poder emplearlos en la obtención de nuestros modelos estadísticos cuando queramos interpretar los resultados.  

In [14]:
Incidents_F.to_csv('Incidents.csv')

In [15]:
import json
import numpy as np

def convert_keys_to_strings(dictionary):
    new_dict = {}
    for key, value in dictionary.items():
        if isinstance(key, np.datetime64):
            key = str(key)
        if isinstance(value, dict):
            value = convert_keys_to_strings(value)
        new_dict[str(key)] = value
    return new_dict

val_dict_str = convert_keys_to_strings(val_dict)


with open('val_dict.json', 'w') as file:
    json.dump(val_dict_str, file)


with open('val_dict.json', 'r') as file:
    val_dict_loaded = json.load(file)


for col, values in val_dict_loaded.items():
    print(f"{col}: {values}")



incident_day_of_week: {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
incident_category: {'FRAUD': 0, 'RECOVERED VEHICLE': 1, 'ASSAULT': 2, 'LARCENY THEFT': 3, 'DISORDERLY CONDUCT': 4, 'MOTOR VEHICLE THEFT': 5, 'MISCELLANEOUS INVESTIGATION': 6, 'BURGLARY': 7, 'OTHER MISCELLANEOUS': 8, 'NON-CRIMINAL': 9, 'MISSING PERSON': 10, 'MALICIOUS MISCHIEF': 11, 'WEAPONS OFFENSE': 12, 'ARSON': 13, 'FIRE REPORT': 14, 'WARRANT': 15, 'SUSPICIOUS OCC': 16, 'OTHER OFFENSES': 17, 'FORGERY AND COUNTERFEITING': 18, 'ROBBERY': 19, 'OTHER': 20, 'TRAFFIC VIOLATION ARREST': 21, 'LOST PROPERTY': 22, 'DRUG OFFENSE': 23, 'EMBEZZLEMENT': 24, 'COURTESY REPORT': 25, 'OFFENCES AGAINST THE FAMILY AND CHILDREN': 26, 'WEAPONS CARRYING ETC': 27, 'STOLEN PROPERTY': 28, 'SEX OFFENSE': 29, 'CASE CLOSURE': 30, 'VANDALISM': 31, 'TRAFFIC COLLISION': 32, 'SUICIDE': 33, 'RAPE': 34, 'VEHICLE IMPOUNDED': 35, 'VEHICLE MISPLACED': 36, 'DRUG VIOLATION': 37, 'SUSPICIOUS': 38, 'HOMIC

A continuación como realizaremos un clustering para explorar nuestro conjunto de datos, observar patrones y tomar decisiones adecuadas necesitamos que todas nuestras variables sean de tipo categórico por ello las variables de tipo numérico day, month, year , hour y minutes se han de convertir en tipo categórico. 

Cómo en el modelo estadístico final vamos a predecir la fecha, parte de las relaciones que tengan esta variable serán capturados por este modelo, por ello la codificación de estas variables será en rangos y además crearemos la variable la variable estación y así podemos determinar si existe alguna relación entre la época del año y los incidentes que se producen. 

La codificación de estas variables se realizará de la siguiente forma:

- Day: cuatro posibles valores; week1, week2, week3, week4. 
- Month: cuatro posibles valores; quarter1, quarter2, quarter3, quarter4. 
- Year: no codificaremos esta variable porque no voy a aplicar el clustering sobre todos los años y seleccionaré únicamente 3 años para reducir el tiempo de cálculo computacional. 
- Hour : seis posibles valores; 0-4, 4-8, 8-12, 12-16, 16-20, 20-24.
- Minutes: cuatro posibles valores; 0-15, 15-30, 30-45, 45-60. 

Finalmente mostramos las columnas y los primeros registros para asegurarnos que todo proceso realizado en esta parte del proyecto es correcto. 

In [16]:
Incidents.columns

Index(['incident_datetime', 'incident_day_of_week', 'incident_code',
       'incident_category', 'incident_subcategory', 'resolution', 'latitude',
       'longitude', 'neighborhood', 'Police_district', 'Holiday', 'Street',
       'day', 'month', 'year', 'hour', 'minutes', 'week', 'quarter', 'season',
       'interval_hour', 'interval_minutes', 'incident_day_of_week_cod',
       'incident_category_cod', 'incident_subcategory_cod', 'resolution_cod',
       'neighborhood_cod', 'Police_district_cod', 'Holiday_cod', 'Street_cod',
       'week_cod', 'quarter_cod', 'season_cod', 'interval_hour_cod',
       'interval_minutes_cod'],
      dtype='object')

In [17]:
Incidents.head()

,incident_datetime,incident_day_of_week,incident_code,incident_category,incident_subcategory,resolution,latitude,longitude,neighborhood,Police_district,...,resolution_cod,neighborhood_cod,Police_district_cod,Holiday_cod,Street_cod,week_cod,quarter_cod,season_cod,interval_hour_cod,interval_minutes_cod
0,2020-06-22 08:00:00,Monday,9029,FRAUD,FRAUD,Open or Active,37.772618,-122.483119,Outer Richmond,RICHMOND,...,0,0,0,0,0,0,0,0,0,0
1,2020-06-22 13:30:00,Monday,7043,RECOVERED VEHICLE,RECOVERED VEHICLE,Open or Active,37.765117,-122.418579,Mission,MISSION,...,0,1,1,0,1,0,0,0,1,1
2,2020-06-22 22:26:00,Monday,12026,ASSAULT,AGGRAVATED ASSAULT,Open or Active,37.728942,-122.378542,Bayview Hunters Point,BAYVIEW,...,0,2,2,0,2,0,0,0,2,2
3,2020-06-22 18:00:00,Monday,6243,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.712320,-122.456045,Oceanview/Merced/Ingleside,TARAVAL,...,0,3,3,0,3,0,0,0,3,0
4,2020-06-22 11:30:00,Monday,6244,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.806658,-122.420560,Russian Hill,CENTRAL,...,0,4,4,0,4,0,0,0,0,1


Una vez hemos codificado estas variables, también vamos a normalizarlas para poder emplearlas en modelos estadísticos sin que las variables con valores más grandes predominen sobre las demás. 
Normalizaremos las variables  'day', 'month', 'hour', 'minutes'. La variable year la podemos tratar como categórica ya que solo toma 6 valores de 2028 a 2023.

In [18]:
Incidents.columns

Index(['incident_datetime', 'incident_day_of_week', 'incident_code',
       'incident_category', 'incident_subcategory', 'resolution', 'latitude',
       'longitude', 'neighborhood', 'Police_district', 'Holiday', 'Street',
       'day', 'month', 'year', 'hour', 'minutes', 'week', 'quarter', 'season',
       'interval_hour', 'interval_minutes', 'incident_day_of_week_cod',
       'incident_category_cod', 'incident_subcategory_cod', 'resolution_cod',
       'neighborhood_cod', 'Police_district_cod', 'Holiday_cod', 'Street_cod',
       'week_cod', 'quarter_cod', 'season_cod', 'interval_hour_cod',
       'interval_minutes_cod'],
      dtype='object')

Así vemos que nuestras variables las que podrán ser tratadas de forma numérica y las que pueden ser tratadas de forma categórica: 

- Numéricas: 'incident_code', 'latitude', 'longitude', 'day', 'month', 'year', 'hour', 'minutes', 'incident_day_of_week_cod','incident_category_cod', 'incident_subcategory_cod', 'resolution_cod',
'neighborhood_cod', 'Police_district_cod', 'Holiday_cod', 'Street_cod','week', 'quarter', 'season', 'interval_hour', 'interval_minutes','day_norm', 'month_norm', 'hour_norm', 'minutes_norm'.

- Categóricas: 'incident_datetime', 'incident_day_of_week', 'incident_category', 'incident_subcategory', 'resolution', 'neighborhood', 'Police_district', 'Holiday', 'Street',



Finalmente, ya hemos terminado el preprocesado de los datos y ya podemos pasar a aplicar modelos estadísticos. En la próxima parte loq ue haremos será crear las muestras a emplear como base para entrenar nuestros modelos.


In [19]:
missing_values_count = Incidents.isna().sum()
print(missing_values_count)


incident_datetime           0
incident_day_of_week        0
incident_code               0
incident_category           0
incident_subcategory        0
resolution                  0
latitude                    0
longitude                   0
neighborhood                0
Police_district             0
Holiday                     0
Street                      0
day                         0
month                       0
year                        0
hour                        0
minutes                     0
week                        0
quarter                     0
season                      0
interval_hour               0
interval_minutes            0
incident_day_of_week_cod    0
incident_category_cod       0
incident_subcategory_cod    0
resolution_cod              0
neighborhood_cod            0
Police_district_cod         0
Holiday_cod                 0
Street_cod                  0
week_cod                    0
quarter_cod                 0
season_cod                  0
interval_h

In [20]:
Incidents.to_csv('Incidents_Final.csv',index = False)